# Chapter 2 : Getting started with MLflow for DL

## **Setting up MLflow**

### **Setting up MLflow to interact with a remote MLflow server**

In a corporate production environment, MLflow is usually hosted on a cloud server, which could be self-hosted or one of the Databricks' managed services in one of the cloud providers (such as AWS, Azure, or Google Cloud). In those cases, there is a requirement to set up your local development environment so that you can run your ML/DL experiment locally but interact with the MLflow server remotely. Next, we will describe how to do this using environment variables with the help of the following three steps:
1. In a bash shell command-line environment, define three new environment variables if you are using a Databricks-managed MLflow tracking server. The first environment variable is `MLFLOW_TRACKING_URI`, and the assigned value is databricks:
```bash
export MLFLOW_TRACKING_URI=databricks
export DATABRICKS_HOST=https://*******
export DATABRICKS_TOKEN=dapi******
```
2. The second environment variable is `DATABRICKS_HOST`. If your Databricks managed website looks like `https://dbc-*.cloud.databricks.com/`, then that's the value of the `DATABRICKS_HOST` variable (replace `*` with your actual website string).
3. The third environment variable is `DATABRICKS_TOKEN`. Navigate to your Databricks-managed website at `https://dbc-*.cloud.databricks.com/#setting/account`, click on Access Tokens, and then click on Generate New Token. You will see a pop-up window with a Comment field (which can be used to record why this token will be used) and expiration date

### **Implementing our first DL experiment with MLflow autologging**

Let's use the DL sentiment classifier we built previously.

In [1]:
# 1 First we need to import the MLflow module
import mlflow
# This will provide API for logging and loading models
# 2 Just before we run the training code, we need to set up an active experiment using
# mlflow.set_experiment for the current running code
EXPERIMENT_NAME = "dl_model_chapter2"
mlflow.set_experiment(EXPERIMENT_NAME)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print("experiment_id:", experiment.experiment_id)

2025/05/25 10:56:08 INFO mlflow.tracking.fluent: Experiment with name 'dl_model_chapter2' does not exist. Creating a new experiment.


experiment_id: 239468158995223302


This sets an experiment named `dl_model_chapter02` to be the current active experiment. If this experiment does not exist in your current tracking server, it will be created automatically.

#### **ENVIRONMENT VARIABLE**

*Note that you might need to set the tracking server URI using the MLFLOW_TRACKING_URI environment variable before you run your first experiment. If you are using a hosted Databricks server, implement the following*:
```bash
export MLFLOW_TRACKING_URI=databricks
```

*If you are using a local server, then set this environment variable to empty or the default localhost at port number 5000 as follows (note that this is our current section's scenario and assumes you are using a local server):*
```bash
export MLFLOW_TRACKING_URI= http://127.0.0.1:5000
```

In [2]:
!export MLFLOW_TRACKING_URI=http://127.0.0.1:5000

In [3]:
# 3 Next, add one line of code to enable autologging in MLflow:
mlflow.pytorch.autolog()

2025/05/25 10:56:14 WARNING mlflow.utils.autologging_utils: MLflow pytorch autologging is known to be compatible with 1.9.0 <= torch <= 2.6.0, but the installed version is 2.7.0+cu126. If you encounter errors during autologging, try upgrading / downgrading torch to a compatible version, or try upgrading MLflow.


This will allow the default parameters, metrics, and model to be automatically logged to the MLflow tracking server.

### **AUTOLOGGING IN MLFLOW**

*Autologging in MLflow is still in experiment mode (as of version 1.20.2) and might change in the future. Here, we use it to explore the MLflow components since it only requires one line of code to automatically log everything of interest. In the upcoming chapters, we will learn about and implement additional ways to perform tracking and logging in MLflow. Also, note that currently, autologging in MLflow for PyTorch (as of version 1.20.2) only works for the PyTorch Lightning framework, not any arbitrary PyTorch code.*

In [4]:
import torch
from torchinfo import summary
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW
from torch.nn.functional import cross_entropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import os
import zipfile
import requests
from tqdm.auto import tqdm


# --- 1. Data Download Utility (similar to Flash's download_data) ---
def download_and_extract_zip(url, path):
    if not os.path.exists(path):
        os.makedirs(path)
    zip_file_name = os.path.join(path, "downloaded_data.zip")

    print(f"Downloading data from {url}...")
    response = requests.get(url, stream=True)
    total_size_in_bytes = int(response.headers.get('content-length', 0))
    block_size = 1024 # 1 Kibibyte
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
    with open(zip_file_name, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()

    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
        print("ERROR, something went wrong during download")
        return

    print(f"Extracting {zip_file_name} to {path}...")
    with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
        zip_ref.extractall(path)
    os.remove(zip_file_name) # Clean up the zip file
    print("Download and extraction complete.")

# --- 2. Custom Dataset for IMDB (replaces TextClassificationData) ---
class IMDBDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Ensure 'sentiment' is mapped to numerical labels
        # 0 for negative, 1 for positive
        self.label_map = {'negative': 0, 'positive': 1}
        self.dataframe['sentiment_label'] = self.dataframe['sentiment'].map(self.label_map)


    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        review = str(self.dataframe.loc[idx, 'review'])
        label = self.dataframe.loc[idx, 'sentiment_label']

        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt', # Return PyTorch tensors
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }



In [5]:
# --- 3. Training Function ---
# """
# Once we have the data, we can now perform fine-tuning using a foundation model.
# First, we declare classifier_model by calling TextClassifier with a backbone
# assigned to prajjwal1/bert-tiny (which is a much smaller BERT-like pretrained
# model located in the Hugging Face model repository: https://huggingface.co/prajjwal1/bert-tiny).
# This means our model will be based on the bert-tiny model.
# """
def train_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(tqdm(dataloader, desc="Training")):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    return total_loss / len(dataloader)

# --- 4. Evaluation Function ---
def evaluate_epoch(model, dataloader, device):
    model.eval()
    total_loss = 0
    all_predictions = []
    all_true_labels = []

    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(dataloader, desc="Evaluating")):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1).cpu().numpy()
            all_predictions.extend(predictions)
            all_true_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(all_true_labels, all_predictions)
    # Get precision, recall, f1-score for positive class (label 1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_true_labels, all_predictions, average='binary', pos_label=1
    )

    return avg_loss, accuracy, precision, recall, f1

In [6]:
# --- 1. & 2. Data Download and Preparation ---
data_url = "https://pl-flash-data.s3.amazonaws.com/imdb.zip"
data_path = "./data/"
download_and_extract_zip(data_url, data_path)

train_df = pd.read_csv(os.path.join(data_path, "imdb/train.csv"))
val_df = pd.read_csv(os.path.join(data_path, "imdb/valid.csv"))
test_df = pd.read_csv(os.path.join(data_path, "imdb/test.csv"))

# Determine number of classes from the sentiment column
num_classes = train_df['sentiment'].nunique()
print(f"Number of classes: {num_classes}")

# Initialize Tokenizer
model_name = "prajjwal1/bert-tiny" # As used in Flash example
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_token_length = 128 # A common max length for BERT-tiny, adjust as needed

# Create custom datasets
train_dataset = IMDBDataset(train_df, tokenizer, max_token_length)
val_dataset = IMDBDataset(val_df, tokenizer, max_token_length)
test_dataset = IMDBDataset(test_df, tokenizer, max_token_length)

# Create DataLoaders
batch_size = 128
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# --- 3. Model Definition and Setup ---
# Use AutoModelForSequenceClassification for classification tasks with Hugging Face models
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)

# Determine device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

# Optimizer and Learning Rate
optimizer = AdamW(model.parameters(), lr=2e-5) # Common learning rate for fine-tuning BERT

# --- 4. Training Loop ---

  0%|          | 0.00/15.9M [00:00<?, ?iB/s]

Extracting ./data/downloaded_data.zip to ./data/...
Download and extraction complete.
Number of classes: 2


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using device: cuda


In [ ]:
num_epochs = 10 # As specified in PyTorch Lightning example
learning_rate = 2e-5 # Common learning rate for fine-tuning BERT

with mlflow.start_run() as run:
    params = {
        "model_name": model_name,
        "num_epochs": num_epochs,
        "num_classes": num_classes,
        "train_size": len(train_dataset),
        "val_size": len(val_dataset),
        "metric_function" : "accuracy",
        "device": str(device),
        "num_gpus": torch.cuda.device_count(),
        "batch_size": batch_size,
        "learning_rate": learning_rate,
        "max_token_length": max_token_length,
        "optimizer": type(optimizer).__name__,
    }
    mlflow.log_params(params)
    print(f"Run ID: {run.info.run_id}")

    with open("model_summary.txt", "w") as f:
        f.write(str(summary(model)))
    mlflow.log_artifact("model_summary.txt")


    print("\nStarting training...")
    for epoch in range(num_epochs):
        print(f"---⌛ Epoch {epoch+1}/{num_epochs} ---")
        train_loss = train_epoch(model, train_dataloader, optimizer, device)
        val_loss, val_accuracy, val_precision, val_recall, val_f1 = evaluate_epoch(model, val_dataloader, device)

        print(f"Train Loss: {train_loss:.4f}")
        print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, "
                f"Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}")

    # Save the trained model to MLflow.
    mlflow.pytorch.log_model(model, artifact_path="model")

    # --- 5. Testing ---
    # Once the fine-tuning step is completed, we will test the accuracy
    # of the model by running trainer.test():
    print("\nStarting testing...")
    test_loss, test_accuracy, test_precision, test_recall, test_f1 = evaluate_epoch(model, test_dataloader, device)
    print(f"Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}, "
            f"Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}")

# --- 6. Prediction Example ---
print("\nExample Prediction on new data:")
model.eval()
new_reviews = [
    "This movie was absolutely fantastic! I loved every moment of it.",
    "Utterly disappointing. A complete waste of time.",
    "It was okay, nothing special.",
    "The acting was superb, but the plot was a bit weak."
]

for review in new_reviews:
    encoding = tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=max_token_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_label_id = torch.argmax(logits, dim=-1).item()
        predicted_sentiment = "positive" if predicted_label_id == 1 else "negative"
        print(f"Review: '{review}' -> Predicted Sentiment: {predicted_sentiment}")

Run ID: b1355d743ccb49d3b362a521ce65c826

Starting training...
---⌛ Epoch 1/10 ---


Training:   0%|          | 0/176 [00:00<?, ?it/s]

We can put all the code into a file named `first_dl_with_mlflow.py` and run the command:
```bash
python first_dl_with_mlflow.py
```

6. Now, we can open the MLflow UI locally to see what has been logged in the local tracking server by navigating to `http://127.0.0.1:5000/`. Here, you will see that a new experiment (`dl_model_chapter02`) with a new run (`Run Name = chapter02`) has been logged:

## **Exploring MLflow's components and usage patterns**

### **Exploring experiments and runs in MLflow**

**Experiment** is a **first-class entity in the MLflow APIs**. **This makes sense as data scientists and ML engineers need to run lots of experiments** in order to build a working model that meets the requirements. However, **the idea of an experiment goes beyond just the model development stage and extends to the entire life cycle of the ML/DL development and deployment**. So, this means that when we do retraining or training for a production version of the model, we need to treat them as production-quality experiments. This unified view of experiments builds a bridge between the offline and online production environments. Each experiment consists of many runs where you can either **change the model parameters**, **input data**, or even **model type** for each run. So, an experiment is an umbrella entity containing a series of runs.

Now, let's explore the MLflow experiments in a hands-on way. Run the following MLflow command line to interact with the tracking server:
```bash
mlflow experiments search
```
If your `MLFLOW_TRACKING_URI` environment variable points to a remote tracking server, then it will list all the experiments that you have read access to. The command lists the three columns of the experiment property: Experiment Id (an integer), Name (a text field that can be used to describe the experiment name), and Artifact Location (by default, this is located in the mlruns folder underneath the directory where you execute the MLflow commands). The mlruns folder is used by a filesystem-based MLflow tracking server to store all the metadata of experiment runs and artifacts.

### **THE COMMAND-LINE INTERFACE (CLI) VERSUS REST APIS VERSUS PROGRAMMING LANGUAGE-SPECIFIC APIS**

MLflow provides three different types of tools and APIs to interact with the tracking server. Here, the CLI is used so that we can explore the MLflow components.

So, let's explore a specific MLflow experiment, as follows:

1. First, create a new experiment using the MLflow CLI, as follows:
```bash
mlflow experiments create -n dl_model_chapter02
```
The preceding command creates a new experiment named `dl_model_chapter02`. If you have already run the first DL model with MLflow autologging in the previous section, the preceding command will cause an error message
2. Now, let's examine the relationship between experiments and runs. If you look carefully at the URL of the run page you will see something similar to the following:
`http://127.0.0.1:5000/#/experiments/1/runs/2f2ec6e72a5042f891abe0d3a533eec7` As you might have gathered, the integer after the experiments path is the experiment ID. Then, after the experiment ID, there is a runs path, followed by a GUID-like random string, which is the run ID. So, now we understand how the runs are organized under the experiment with a globally unique ID (called a run ID).

Knowing a run's globally unique ID is very useful. This is because we can retrieve the run's logged data using `run_id`. If you use the mlflow runs describe `--run-id <run_id>` command line, you can get the list of metadata that this run has logged. For the experiment we just ran, the following shows the full command with the run ID:
```bash
mlflow runs describe –-run-id 9b1034c2b4c54125ae78705b62514b1c
```

This will present all the metadata about this run in JSON format. This metadata includes parameters used by the model training; metrics for measuring the accuracy of the model in training, validation, and testing; and more. The same data is also presented in the MLflow UI. Note that the powerful MLflow CLI will allow very convenient exploration of the MLflow logged metadata and artifacts as well as enabling shell script-based automation, as we will explore in the upcoming chapters.

## **Exploring MLflow models and their usages**

### **THE MLFLOW TRACKING SERVER'S BACKEND STORE AND ARTIFACT STORE**

An MLflow tracking server has two types of storage: first, a backend store, which stores experiments and runs metadata along with params, metrics, and tags for runs; and second, an artifact store, which stores larger files such as serialized models, text files, or even generated plots for visualizing model results. For the purpose of simplicity, in this chapter, we are using a local filesystem for both the backend store and the artifact store. However, some of the more advanced features such as model registry are not available in a filesystem-based artifact store. In later chapters, we will learn how to use a model registry.

Let's look at the list of artifacts, one by one:
* `model_summary.txt`: At the root folder level, this file looks similar to the following output if you click on it. It describes the model metrics and the layers of the DL model and provides a quick overview of what the DL model looks like in terms of the number and type of neural network layers, the number and size of the parameters, and the type of metrics used in training and validation. This is very helpful when the DL model architecture is needed to be shared and communicated among team members or stakeholders.
* The `model` folder: This folder contains a subfolder, called data, and three files called `MLmodel`, `conda.yaml`, and `requirements.txt`:
  * `MLmodel`: This file describes the flavor of the model that MLflow supports. Flavor is MLflow-specific terminology. It describes how the model is saved, serialized, and loaded. For our first DL model, the following information is stored in an `MLmodel` file
  * `conda.yaml`: This is a conda environment definition file used by the model to describe our dependencies
  * `requirements.txt`: This is a Python pip-specific dependency definition file. It is just like the pip section in the conda.yaml file, as shown in Figure 2.16.
data: This is a folder that contains the actual serialized model, called model.pth, and a description file, called pickle_module_info.txt, whose content for our first DL experiment is as follows: `mlflow.pytorch.pickle_module`

This means the model is serialized using a PyTorch-compatible pickle serialization method provided by MLflow. This allows MLflow to load the model back to memory later if needed

### **MODEL REGISTRY VERSUS MODEL LOGGING**

The MLflow model registry requires a relational database such as MySQL as the artifact store, not just a plain filesystem. Therefore, in this chapter, we will not explore it yet. Note that a model registry is different from model logging in that, for each run, you want to log model metadata and artifacts. However, only for certain runs that meet your production requirements, you may want to register them in the model registry for production deployment and version control. In later chapters, we will learn how to register models.

By now, you should have a good understanding of the list of files and metadata about the model and the serialized model (along with the .pth file extension in our experiment, which refers to a PyTorch serialized model) logged in the MLflow artifact store.

## **Exploring MLflow code tracking and its usages**

When exploring the metadata of the run, we can also discover how the code is being tracked. As shown in the MLflow UI and the command-line output in JSON, the code is tracked in three ways: a filename, a Git commit hash, and a source type. You can execute the following command line:

```bash
mlflow runs describe --run-id 9b1034c2b4c54125ae78705b62514b1c | grep mlflow.source
```
Based on this 9b1034c2b4c54125ae78705b62514b1c Git commit hash, we can go on to find the exact copy of the Python code we used.

Note that, here, the source type is `LOCAL`. This means that we execute the MLflow-enabled source code from a local copy of the code.

### **LOCAL VERSUS REMOTE GITHUB CODE**

If the source is a local copy of the code, there is a caveat regarding the Git commit hash that you see in the MLflow tracking server. If you make code changes locally but forget to commit them and then immediately start an MLflow experiment tracking run, MLflow will only log the most recent Git commit hash. We can solve this problem in one of two ways:

1. Commit our code changes before running the MLflow experiment
2. Use remote GitHub code to run the experiment.

Since the first method is not easily enforceable, the second method is preferred. Using remote GitHub code to run a DL experiment is an advanced topic that we will explore in later chapters.

So far, we have learned about the MLflow tracking server, experiments, and runs. Additionally, we have logged metadata about runs such as parameters and metrics, examined code tracking, and explored model logging. These tracking and logging capabilities ensure that we have a solid ML experiment management system, not only for model development but also for model deployment in the future, as we need to track which runs produce the model for production. Reproducibility and provenance-tracking are the hallmarks of what MLflow provides. In addition to this, MLflow provides other components such as MLproject for standardized ML project code organization, a model registry for model versioning control, model deployment capabilities, and model explainability tools.